# **Database Access**

+ Data are observations or measurements (unprocessed or processed) represented as text, numbers, or multimedia.
+ A dataset is a structured collection of data generally associated with a unique body of work.
+ A database is an organized collection of data stored as multiple datasets.

# **SQLite**
+ SQLite is a C library that provides a lightweight disk-based database that can provide a relational database management system with zero-configuration.
+ SQLite is an embedded SQL database engine. Unlike most other SQL databases, SQLite does not have a separate server process.
+ The SQLite project was started on 2000-05-09.
+ SQLite database files are commonly used as containers to transfer rich content between systems.
+ SQLite is built into all mobile phones and most computers.
+ It can be used in many programming languages including Python
+ We will use SQLite version 3 or SQLite3


**In summary**:
SQLite is a library used for quick and simple databases.

+ quick: we dont need to setup an special data server for this :) - It is a serverless database
+ simple: because it uses regular SQL commands (English instructions alike)



## **Create Connection and Database**
+ First import the sqlite3 module and then create a connection object which will connect us to the database.
+ Call sqlite3.connect() to create a connection to the database example.db in the current working directory.

+ That will create a new file with the name ‘mydatabase.db’.
+ The returned Connection object con represents the connection to the on-disk database.


In [71]:
import sqlite3

con = sqlite3.connect('mydatabase.db')


In [72]:
con

## **SQLite3 Cursor**
+ In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor.
+ The SQLite3 cursor is a method of the connection object.
+ Call con.cursor() to create the Cursor as follow:
+ Now we can use the cursor object to call the execute() method to execute any SQL queries.

In [73]:
cursorObj = con.cursor()

In [74]:
cursorObj

## **Create a Table**
+ To create a table in SQLite3, we can use the CREATE TABLE query in the execute() method. Consider the following steps:
1) Create a connection object.
2) From the connection object, create a cursor object.
3) Using the cursor object, call the execute method with create table query as the parameter.
+ Let’s create employees with the following attributes. Execute the CREATE TABLE statement by calling cursorObj.execute(...)
 + (id, name, salary, position)

+ The PRIMARY KEY constraint uniquely identifies each record in a table.
+ Specifying the data types is optional.


In [75]:
cursorObj.execute("CREATE TABLE employees(id integer PRIMARY KEY, name text, salary real, position text)")


## **Verify Table**
+ We can verify that the newly created table by querying the sqlite_master table built-in to SQLite, which should now contain an entry for the employees table definition.
+ Execute that query by calling cursorObj.execute(...), assign the result to res, and call res.fetchone() to fetch the resulting row:




In [76]:
res = cursorObj.execute("SELECT name FROM sqlite_master")
res.fetchone()[0]

'employees'

+ If we query sqlite_master for a non-existent table, res.fetchone() will return None:

In [77]:
res = cursorObj.execute("SELECT name FROM sqlite_master WHERE name='Cat' ")
res.fetchone() is None

True

In [ ]:
## Alternative option to verify a table
cursorObj.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursorObj.fetchall())

In [ ]:
names = [description[0] for description in cursorObj.description]
names

In [ ]:
cursorObj.description

In [78]:
def sqlite_table_schema(conn, name):
    cursor = conn.execute("SELECT sql FROM sqlite_master WHERE name=?;", [name])
    sql = cursor.fetchone()[0]
    cursor.close()
    return sql

In [79]:
print(sqlite_table_schema(con, 'employees'))

CREATE TABLE employees(id integer PRIMARY KEY, name text, salary real, position text)


## **Insert in Table**
+ To insert data in a table, we use the INSERT INTO statement. Consider the following line of code:


In [80]:
cursorObj.execute("INSERT INTO employees VALUES(1, 'Mike', 1000, 'Manager') ")
con.commit()

+ The INSERT statement implicitly opens a transaction, which needs to be committed before changes are saved in the database (see Transaction control for details). Call con.commit() on the connection object to commit the transaction.
+ We can verify that the data was inserted correctly by executing a SELECT query. Use the cursorObj.execute(...) to assign the result to res, and call res.fetchall() to return all resulting rows:


In [84]:
res = cursorObj.execute("SELECT name FROM employees")
res.fetchall()

[('Mike',)]

In [86]:
res = cursorObj.execute("SELECT * FROM employees")
res.fetchone()

(1, 'Mike', 1000.0, 'Manager')

## **Insert more rows in Table**

 + We can insert three more rows by calling cur.executemany(...):
+ We can use the question mark (?) as a placeholder for each value.


In [87]:
data = [
    (2, 'Jack', 850, 'IT'),
    (3, 'Andrew', 800, 'Tech'),
    (4, 'Mason', 1500, 'CTO'),
    (5, 'Abel', 1000, 'CTO'),
]


In [88]:
cursorObj.executemany("INSERT INTO employees VALUES(?, ?, ?,?)", data)
con.commit()  # Remember to commit the transaction after executing INSERT.


+ Verify that the new rows by executing a SELECT query:


In [91]:
#for row in cursorObj.execute("SELECT name, salary, position FROM employees ORDER BY salary"):
for row in cursorObj.execute("SELECT name, salary FROM employees ORDER BY salary"):
    print(row)


('Andrew', 800.0)
('Jack', 850.0)
('Mike', 1000.0)
('Abel', 1000.0)
('Mason', 1500.0)


## **Update Table**
+ To update the table, simply create a connection, then create a cursor object using the connection and finally use the UPDATE statement in the execute() method.
+ Suppose we want to update the employee’s salary whose id equals 3, we will use the UPDATE statement for the employee whose id equals 3.
+ The WHERE clause will be used as a condition to select this employee.

In [92]:
cursorObj.execute('UPDATE employees SET salary = 900 where id = 3')
con.commit()


+ We now can verify the updated table by the statement:


In [93]:
for row in cursorObj.execute("SELECT name, salary, position FROM employees order by salary"):
    print(row)


('Jack', 850.0, 'IT')
('Andrew', 900.0, 'Tech')
('Mike', 1000.0, 'Manager')
('Abel', 1000.0, 'CTO')
('Mason', 1500.0, 'CTO')


## **Fetch data**

+ To fetch specific rows, we can combine SELECT, WHERE, fetchall statements  
+ Suppose we want to print the id and names of those who have a salary greater than 1000:


In [94]:
cursorObj.execute('SELECT id, name FROM employees WHERE salary >= 1000.0')

In [95]:
rows=cursorObj.fetchall()
for row in rows:
  print(row)

(1, 'Mike')
(4, 'Mason')
(5, 'Abel')


# **Select Statement**

In [96]:
for row in cursorObj.execute("SELECT position FROM employees"):
    print(row)


('Manager',)
('IT',)
('Tech',)
('CTO',)
('CTO',)


In [98]:
res = cursorObj.execute("SELECT name FROM employees")
res.fetchall()


[('Mike',), ('Jack',), ('Andrew',), ('Mason',), ('Abel',)]

In [99]:
for row in cursorObj.execute("SELECT * FROM employees"):
    print(row)



(1, 'Mike', 1000.0, 'Manager')
(2, 'Jack', 850.0, 'IT')
(3, 'Andrew', 900.0, 'Tech')
(4, 'Mason', 1500.0, 'CTO')
(5, 'Abel', 1000.0, 'CTO')


# **LIMIT operator**

In [100]:
# SELECT the first 2 rows of employees
cursorObj.execute('SELECT * FROM employees LIMIT 2;')
cursorObj.fetchall()


[(1, 'Mike', 1000.0, 'Manager'), (2, 'Jack', 850.0, 'IT')]

In [103]:
cursorObj.execute('SELECT id, name, salary FROM employees WHERE salary > 900.0')
rows = cursorObj.fetchall()
for row in rows:
    print(row)


(1, 'Mike', 1000.0)
(4, 'Mason', 1500.0)
(5, 'Abel', 1000.0)


In [105]:
cursorObj.execute('SELECT id, name, salary FROM employees WHERE salary > 500.0  LIMIT 2 OFFSET 1')
rows = cursorObj.fetchall()
for row in rows:
    print(row)


(2, 'Jack', 850.0)
(3, 'Andrew', 900.0)


# **OFFSET operator**

In [101]:
# SELECT the first 2 rows of employees starting from third row
cursorObj.execute('SELECT * FROM employees LIMIT 2 OFFSET 2')
cursorObj.fetchall()


[(3, 'Andrew', 900.0, 'Tech'), (4, 'Mason', 1500.0, 'CTO')]

# **LIKE operator**

In [106]:
cursorObj.execute('SELECT * from employees WHERE name LIKE "%Ja%" ')
cursorObj.fetchall()

[(2, 'Jack', 850.0, 'IT')]

In [107]:
cursorObj.execute('SELECT * from employees WHERE name LIKE "%Ab%" ')
cursorObj.fetchall()

[(5, 'Abel', 1000.0, 'CTO')]

In [108]:
cursorObj.execute('SELECT * from employees WHERE salary LIKE "1%" ')
cursorObj.fetchall()


[(1, 'Mike', 1000.0, 'Manager'),
 (4, 'Mason', 1500.0, 'CTO'),
 (5, 'Abel', 1000.0, 'CTO')]